In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
print("TensorFlow version:", tf.__version__)
import math
import matplotlib.pyplot as plt
import wfdb
import ast
import sys
import matplotlib.pyplot as plt


def get_freq_bins(signal,fs,num_bins,bin_width,verbose=False):
#function that returns num_bins number of bins with width bin_width 
    size = len(signal)
    ts = 1/fs
    amplidtude = np.fft.fft(signal,size)/size
    df=fs/size; #frequency resolution
    f=[i*df for i in range(0,int(size/2))]
    if verbose:
        plt.figure(1)
        plt.subplot(2,1,1)
        plt.stem(f,abs(frequencyArr[0:int(size/2)]))
        plt.subplot(2,1,2)
        plt.plot(signal)

    bin_array = np.array(0,dtype = float)
    
    for i in range(0,num_bins*bin_width,bin_width):
        #first index of bin array is the amplitudes from fq 1 to 5. second is 5 to 10 and so on
        bin_array = np.append(bin_array, np.sum(abs(amplidtude[i:i+bin_width+1])))
    bin_array = np.delete(bin_array,0)
    #Make graphs of time series and frequency and save those images as .png
    #Make parameter to pass patient ID save as patient_ID_FFT.png patient_ID_TIMESERIES.png
    #add validation dataset
    return bin_array
#use hill climbing to determine the best bin width
df_normal = pd.read_csv("Data\ptbdb_normal.csv")
df_abnormal = pd.read_csv("Data\ptbdb_abnormal.csv")

TensorFlow version: 2.6.0


In [2]:

bins=77
width = 1
width_start = 1
width_finish = 10
best_index = 0
best = 0

In [3]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

#path = 'path/to/ptbxl/'
path = 'Data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
#path = 'D:/OneDrive/SchoolFiles/CSCE/Graduate/"CSCE 5214 - Software for AI"/Project2/Data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass



#pass single signals to get_freq_bins array

#reset indecies of y array because it skips some
y_train = y_train.reset_index()
y_test = y_test.reset_index()


In [4]:
#make single signals out of data
np.set_printoptions(threshold=sys.maxsize)
fft_arr = np.ndarray((len(X_train),bins+1))
for k in range(0,len(X_train)):
    x_long = []
    for j in range(0,1000):
        for i in range(0,12):
            x_long.append(X_train[k][j][i])
    ground_truth = -1
    if y_train.iloc[i][1] == ['NORM']:
        ground_truth = 0
    else:
        ground_truth = 1
    fft_arr[k,0:bins] = get_freq_bins(x_long,100,bins,width)
    fft_arr[k,bins-1] = ground_truth
    
#X_train -> X_test
#y_train -> y_test
#fft_arr -> test_fft_arr
test_fft_arr = np.ndarray((len(X_test),bins+1))

for k in range(0,len(X_test)):
    x_long = []
    for j in range(0,1000):
        for i in range(0,12):
            x_long.append(X_test[k][j][i])
    ground_truth = -1
    if y_test.iloc[i][1] == ['NORM']:
        ground_truth = 0
    else:
        ground_truth = 1
    test_fft_arr[k,0:bins] = get_freq_bins(x_long,100,bins,width)
    test_fft_arr[k,bins-1] = ground_truth


In [5]:

#build n by m+1 array where last index is truth value and m is number of bins
normal_arr = np.zeros((1,bins+1))
for i in range(0,len(df_normal)):
    temp_train = get_freq_bins(np.asarray(df_normal.iloc[i,:]),125,bins,width)
    normal_arr = np.vstack([normal_arr, np.append(temp_train,0)])

abnormal_arr = np.zeros((1,bins+1))
for i in range(0,len(df_abnormal)):
    temp_train = get_freq_bins(np.asarray(df_abnormal.iloc[i,:]),125,bins,width)
    abnormal_arr = np.vstack([abnormal_arr, np.append(temp_train,1)])

#make testing sets
normal_test = normal_arr[int(len(normal_arr)*0.9):]
abnormal_test = abnormal_arr[int(len(abnormal_arr)*0.9):]
normal_train = normal_arr[0:int(len(normal_arr)*0.9)]
abnormal_train = abnormal_arr[0:int(len(abnormal_arr)*0.9)]

#combine both classes
train= np.append(abnormal_train, normal_train,axis=0)
test= np.append(abnormal_test, normal_test,axis=0)

train= np.append(train, fft_arr,axis=0)
test= np.append(test, test_fft_arr,axis=0)


## LSTM Modification
train_len = len(train)
test_len = len(test)
train = train.reshape((train.shape[0],train.shape[1],1))
test = test.reshape((test.shape[0],test.shape[1],1))




In [6]:
print(type(train))

<class 'numpy.ndarray'>


In [12]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.Dense(bins, activation='relu'),
    
    #tf.keras.layers.LSTM(bins,input_shape=train.shape[1:]),#######################################################
    tf.keras.layers.LSTM(bins,input_shape=((77, 1))),#######################################################

    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

#model.fit(x=train[:,0:bins-1],y=train[:,bins],epochs = 100,verbose=0)#training, not test
model.fit(x=train[:,0:bins],y=train[:,bins],epochs = 100,verbose=0)#training, not test
#add validation dataset
#get confidence of classification

loss, accuracy = model.evaluate(x=test[:,0:bins],y=test[:,bins], verbose=2)#test
if(accuracy > best):
    best_index = bins
    best = accuracy
    model.save('./bins_model.h5')

115/115 - 1s - loss: 0.0261 - accuracy: 0.9907


In [10]:
#in order to make the LSTM work, i have to make each input 2D (total 3D data). 
train.shape[1:]

(78, 1)

In [13]:
model.save('./CAD_LSTM_11_7.h5')

In [15]:
test[1,0:bins]

array([[0.14036196],
       [0.07787759],
       [0.06111694],
       [0.02790608],
       [0.05024123],
       [0.05529711],
       [0.05286239],
       [0.04424322],
       [0.04080974],
       [0.04309676],
       [0.04017423],
       [0.05009808],
       [0.03831543],
       [0.04447626],
       [0.02898949],
       [0.03231368],
       [0.03353185],
       [0.03030187],
       [0.03612139],
       [0.03000473],
       [0.0363709 ],
       [0.02825955],
       [0.03216185],
       [0.02215148],
       [0.02189557],
       [0.02770351],
       [0.02167511],
       [0.02686774],
       [0.02237765],
       [0.02270548],
       [0.01652249],
       [0.01683325],
       [0.01467478],
       [0.01360304],
       [0.01795416],
       [0.01370327],
       [0.01515558],
       [0.00970601],
       [0.00980017],
       [0.0093999 ],
       [0.00730932],
       [0.00868041],
       [0.00686135],
       [0.0091971 ],
       [0.00642492],
       [0.00756877],
       [0.00733209],
       [0.005

In [16]:
test[1,bins]

array([1.])